## Exploring the Dataset

In [66]:
import pandas as pd

file = 'academy_awards.csv'
data = pd.read_csv(file, encoding = 'ISO-8859-1')

#print(data.head())
#pd.value_counts(data['Additional Info'].values, sort=False)

data[:3]



,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN


## Filtering The Data

In [67]:
#Clean the year column by selecting just the first 4 digits
data['Year'] = data['Year'].str[0:4].astype('int64')

#rows from the Dataframe where the Year column is larger than 2000
later_than_2000 = data[data['Year']>2000]

#ows from later_than_2000 where the Category matches one of the 4 awards we're interested in
award_categories = ['Actor -- Leading Role','Actor -- Supporting Role','Actress -- Leading Role','Actress -- Supporting Role']
nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]

nominations[:3]

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN


## Cleaning Up the won? and Unnamed columns

In [68]:
#Replace in the Won? column values by 0/1
replacements = { "NO": 0, "YES": 1 }
nominations_won = nominations.loc[:,'Won?']
nominations_won_cleaned = nominations_won.map(replacements)

nominations['Won'] = nominations_won_cleaned

#Define the column names to be dropped
drop_cols = ["Won?","Unnamed: 5", "Unnamed: 6","Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10"]
final_nominations = nominations.drop(drop_cols, axis=1)


/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [69]:
final_nominations[:3]

,Year,Category,Nominee,Additional Info,Won
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0


## Cleaning Up the Additional Info column

In [70]:
additional_info_one = final_nominations['Additional Info'].str.rstrip("'}")
additional_info_two = additional_info_one.str.split(" {'")
movie_names = additional_info_two.str[0]
characters = additional_info_two.str[1]

final_nominations['Movie'] = movie_names
final_nominations['Character'] = characters

final_nominations = final_nominations.drop("Additional Info", axis=1)
final_nominations.head()


,Year,Category,Nominee,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,0,127 Hours,Aron Ralston


## Exporting to SQLITE3

In [72]:
import sqlite3

#Create and Connect to nomination DB
conn = sqlite3.connect("nominations.db")

#Export final_nominations to nominations.db
final_nominations.to_sql("nominations", conn, index=False)


ValueError: Table 'nominations' already exists.

## Querying

In [ ]:
import sqlite3
conn = sqlite3.connect("nominations.db")

#print the schema of the table
query_one = "pragma table_info(nominations);"

query_two = "select * from nominations limit 10;"
#print(conn.execute(query_one).fetchall())
print(conn.execute(query_two).fetchall())
conn.close()
